In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re


In [16]:
results_dir = 'results'
snapshot_dir = os.path.join(results_dir, 'snapshots')
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    print(f"Created directory: {results_dir}")
if not os.path.exists(snapshot_dir):
    os.makedirs(snapshot_dir)
    print(f"Created directory: {snapshot_dir}")
print("-" * 50)


--------------------------------------------------


In [17]:
file_path = 'venvx/AnnonymData.csv'
rows_for_snapshot = 1000
try:
    # For large files, be mindful of memory.
    # We'll load it directly for now, but consider 'chunksize' or 'dtype' optimization if needed.
    df = pd.read_csv(file_path)
    print(f"Successfully loaded data from '{file_path}'.")
    print(f"Dataset shape: {df.shape}") # (rows, columns)




    snapshot1_path = os.path.join(snapshot_dir, 'snapshot_1_raw_loaded_dataset.html')
    df.head(rows_for_snapshot).to_html(snapshot1_path, escape=False, index=False) # escape=False for cleaner HTML, index=False to not write pandas index
    print(f"\nSnapshot 1: First {rows_for_snapshot} rows of the raw loaded dataset saved to '{snapshot1_path}'")


    print("\nFirst 5 rows of the dataset:")
    print(df.head())



    print("\nDataset info:")
    df.info(verbose=True, show_counts=True)
except FileNotFoundError:
    print(f"ERROR: File not found at '{file_path}'. Please check the path and filename.")
    exit()
except Exception as e:
    print(f"An error occurred during data loading: {e}")
    exit()
print("-" * 50)

Successfully loaded data from 'venvx/AnnonymData.csv'.
Dataset shape: (6538739, 14)

Snapshot 1: First 1000 rows of the raw loaded dataset saved to 'results/snapshots/snapshot_1_raw_loaded_dataset.html'

First 5 rows of the dataset:
    OrderId                     TransactionId DateOfService  \
0  11518978  4c5060636f584ef9a1effa77282755f5    2020-01-02   
1  11285143  68472c70b9c84fb784834ecc257827d7    2020-01-02   
2  11285146  7262eace0d104592b1269e38f5b45ec1    2020-01-02   
3  11285152  8e451931e8fc4554869c3e4533b65e23    2020-01-02   
4  11285155  bfa8fa0812ee40baa98e5aaf52d30e0b    2020-01-02   

           DateOfOrder  OrderQty                     MenuName MenuPrice  \
0  2020-02-05 11:54:08         1             Mittagessen (Gs)      3,10   
1  2019-12-16 10:30:51         1  Smart Eating Buffet (WGrus)      0,00   
2  2019-12-16 10:31:33         1  Smart Eating Buffet (WGrus)      2,90   
3  2019-12-16 10:32:05         1  Smart Eating Buffet (WGrus)      0,00   
4  2019-12-16

In [18]:
print("--- Step 2: Initial Data Cleaning & Preprocessing ---")
df_processed = df.copy()

date_columns = ['DateOfService', 'DateOfOrder', 'DateOfCancel']
for col in date_columns:
    if col in df_processed.columns:
        # Attempt conversion, coercing errors will turn unparseable dates into NaT (Not a Time)
        df_processed[col] = pd.to_datetime(df_processed[col], errors='coerce')
        print(f"Column '{col}' converted. NaNs introduced by coercion: {df_processed[col].isnull().sum()}")
    else:
        print(f"Warning: Date column '{col}' not found.")

--- Step 2: Initial Data Cleaning & Preprocessing ---
Column 'DateOfService' converted. NaNs introduced by coercion: 0
Column 'DateOfOrder' converted. NaNs introduced by coercion: 0
Column 'DateOfCancel' converted. NaNs introduced by coercion: 5604260


In [19]:
print("\nConverting financial columns ('MenuPrice', 'MenuSubsidy') to numeric...")
financial_columns = ['MenuPrice', 'MenuSubsidy']
for col in financial_columns:
    if col in df_processed.columns:
        if df_processed[col].dtype == 'object': # Only process if it's an object type
            # Remove currency symbols (e.g., €, $, £) and commas (as thousands separators)
            # This regex is an example, adjust if your currency format is different
            df_processed[col] = df_processed[col].astype(str).str.replace(r'[€\$£,]', '', regex=True)
            # Convert to numeric, coercing errors.
            # If your numbers use ',' as decimal (e.g., German format), you'd first remove '.', then replace ',' with '.'
            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            print(f"Column '{col}' converted to numeric. NaNs introduced: {df_processed[col].isnull().sum()}")
        elif pd.api.types.is_numeric_dtype(df_processed[col]):
            print(f"Column '{col}' is already numeric.")
        else:
            print(f"Column '{col}' is of type {df_processed[col].dtype} and was not processed as a typical currency string.")
    else:
        print(f"Warning: Financial column '{col}' not found.")


Converting financial columns ('MenuPrice', 'MenuSubsidy') to numeric...
Column 'MenuPrice' converted to numeric. NaNs introduced: 0
Column 'MenuSubsidy' converted to numeric. NaNs introduced: 0


In [20]:
print("\nEnsuring 'OrderQty' and 'CanceledQty' are numeric...")
for col in ['OrderQty', 'CanceledQty']:
    if col in df_processed.columns:
        if not pd.api.types.is_numeric_dtype(df_processed[col]):
            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
            print(f"Column '{col}' converted to numeric. NaNs introduced: {df_processed[col].isnull().sum()}")
        else:
            print(f"Column '{col}' is already numeric.")
    else:
        print(f"Warning: Column '{col}' not found.")


Ensuring 'OrderQty' and 'CanceledQty' are numeric...
Column 'OrderQty' is already numeric.
Column 'CanceledQty' is already numeric.


In [21]:
if 'Site' in df_processed.columns:
    missing_site_percentage = df_processed['Site'].isnull().mean() * 100
    print(f"\nMissing values in 'Site': {missing_site_percentage:.2f}%")
    # Example: df_processed['Site'].fillna('Unknown', inplace=True)
    # Or, if you know the main sites are MS, LP, BK, you might investigate if missing sites can be inferred.
    # For now, let's fill with 'Unknown' as a placeholder strategy.
    df_processed['Site'] = df_processed['Site'].fillna('UnknownSite')
    print("Filled missing 'Site' values with 'UnknownSite'.")


Missing values in 'Site': 12.94%
Filled missing 'Site' values with 'UnknownSite'.


In [22]:
print("\nFiltering out irrelevant orders...")
initial_rows = len(df_processed)

# Condition 1: Orders where DateOfCancel is after DateOfService
if 'DateOfCancel' in df_processed.columns and 'DateOfService' in df_processed.columns:
    condition1_filter = (df_processed['DateOfCancel'].notna()) & \
                        (df_processed['DateOfService'].notna()) & \
                        (df_processed['DateOfCancel'] > df_processed['DateOfService'])
    rows_to_drop_cond1 = df_processed[condition1_filter]
    if not rows_to_drop_cond1.empty:
        print(f"Found {len(rows_to_drop_cond1)} orders where DateOfCancel is after DateOfService. These will be dropped.")
        df_processed = df_processed[~condition1_filter]
    else:
        print("No orders found where DateOfCancel is after DateOfService.")


Filtering out irrelevant orders...
Found 70970 orders where DateOfCancel is after DateOfService. These will be dropped.


In [23]:
# Condition 2: Orders where OrderQty < CanceledQty
# Ensure both columns are numeric and handle potential NaNs before comparison
if 'OrderQty' in df_processed.columns and 'CanceledQty' in df_processed.columns:
    # Fill NaNs with 0 for comparison, assuming NaN in Qty means 0 for this specific filter logic
    order_qty_filled = df_processed['OrderQty'].fillna(0)
    canceled_qty_filled = df_processed['CanceledQty'].fillna(0)

    condition2_filter = order_qty_filled < canceled_qty_filled
    rows_to_drop_cond2 = df_processed[condition2_filter]
    if not rows_to_drop_cond2.empty:
        print(f"Found {len(rows_to_drop_cond2)} orders where OrderQty < CanceledQty. These will be dropped.")
        df_processed = df_processed[~condition2_filter]
    else:
        print("No orders found where OrderQty < CanceledQty.")

rows_after_filtering = len(df_processed)
print(f"Rows dropped due to filtering: {initial_rows - rows_after_filtering}")
print(f"Dataset shape after filtering: {df_processed.shape}")


Found 793547 orders where OrderQty < CanceledQty. These will be dropped.
Rows dropped due to filtering: 864517
Dataset shape after filtering: (5674222, 14)


In [24]:
# f. Create the Target Variable: `NeededMeals`
# NeededMeals = OrderQty - CanceledQty (after filtering and ensuring NaNs in Qty are handled)
print("\nCreating the target variable 'NeededMeals'...")
if 'OrderQty' in df_processed.columns and 'CanceledQty' in df_processed.columns:
    # Assuming that if CanceledQty is NaN for an order that wasn't filtered out, it means 0 cancellations for that order.
    # And if OrderQty is NaN (should be rare after initial checks), treat as 0 for this calculation.
    df_processed['OrderQty_filled'] = df_processed['OrderQty'].fillna(0)
    df_processed['CanceledQty_filled'] = df_processed['CanceledQty'].fillna(0)

    df_processed['NeededMeals'] = df_processed['OrderQty_filled'] - df_processed['CanceledQty_filled']

    # Clean up temporary columns
    df_processed.drop(columns=['OrderQty_filled', 'CanceledQty_filled'], inplace=True)

    print("'NeededMeals' column created.")
    print("Summary of 'NeededMeals':")
    print(df_processed['NeededMeals'].describe())

    # Sanity check: NeededMeals should not be negative if OrderQty < CanceledQty was filtered.
    # However, if OrderQty was 0 and CanceledQty was 0, NeededMeals is 0.
    # If OrderQty was 0 and CanceledQty was 1 (bank account not covered), this record should have been filtered.
    if not df_processed[df_processed['NeededMeals'] < 0].empty:
        print(f"Warning: Found {len(df_processed[df_processed['NeededMeals'] < 0])} records with negative NeededMeals. Review filtering logic.")
        print(df_processed[df_processed['NeededMeals'] < 0][['OrderQty', 'CanceledQty', 'NeededMeals']].head())
    else:
        print("No negative 'NeededMeals' found after calculation, which is good.")
else:
    print("Could not create 'NeededMeals' as 'OrderQty' or 'CanceledQty' is missing.")


Creating the target variable 'NeededMeals'...
'NeededMeals' column created.
Summary of 'NeededMeals':
count    5.674222e+06
mean     9.985110e-01
std      4.098289e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.200000e+02
Name: NeededMeals, dtype: float64
No negative 'NeededMeals' found after calculation, which is good.


In [25]:
print("\n--- Addressing 'MenuName' Variations (Initial Pass) ---")
if 'MenuName' in df_processed.columns:
    unique_menu_names_count_before = df_processed['MenuName'].nunique()
    print(f"Number of unique 'MenuName' entries before any cleaning: {unique_menu_names_count_before}")

    # Perform basic cleaning first
    df_processed['MenuName_Cleaned'] = df_processed['MenuName'].astype(str).str.lower().str.strip()
    unique_menu_names_count_after_basic = df_processed['MenuName_Cleaned'].nunique()
    print(f"Number of unique 'MenuName_Cleaned' entries after basic cleaning (lowercase, strip): {unique_menu_names_count_after_basic}")

    if unique_menu_names_count_after_basic > 50:
        print("First 20 unique 'MenuName_Cleaned' (sample after basic cleaning):")
        print(df_processed['MenuName_Cleaned'].unique()[:20])
    else:
        print("Unique 'MenuName_Cleaned' (after basic cleaning):")
        print(df_processed['MenuName_Cleaned'].unique())

    snapshot5_path = os.path.join(snapshot_dir, 'snapshot_5_after_basic_MenuName_cleaning.html')
    # Displaying head() with the new MenuName_Cleaned column
    df_processed.head(rows_for_snapshot).to_html(snapshot5_path, escape=False, index=False)
    print(f"\nSnapshot 5: First {rows_for_snapshot} rows (showing 'MenuName_Cleaned') saved to '{snapshot5_path}'")
else:
    print("Warning: 'MenuName' column not found.")

print("\n--- Initial Data Cleaning & Preprocessing (Part 1 - Basic Steps) Complete ---")
print("Current DataFrame shape:", df_processed.shape)
print("-" * 50)



--- Addressing 'MenuName' Variations (Initial Pass) ---
Number of unique 'MenuName' entries before any cleaning: 322
Number of unique 'MenuName_Cleaned' entries after basic cleaning (lowercase, strip): 322
First 20 unique 'MenuName_Cleaned' (sample after basic cleaning):
['mittagessen (gs)' 'smart eating buffet (wgrus)' 'mittagessen (bs)'
 'smart eating buffet (primus)' 'menü a (hss)'
 'smart eating buffet (gymbo)' 'smart eating buffet (brs)' 'menü b1'
 'smart eating buffet (ema)' 'mittagessen (ml)'
 'smart-eating buffet (egm)' 'dge-menü (mg)' 'menü a (bo-mitte)'
 'menü a (marbi)' 'smart eating buffet (gma)' 'mittagessen (bk)' 'menü a'
 'menü a (szg)' 'salatbar (mpg)' 'menü b2 (lui)']

Snapshot 5: First 1000 rows (showing 'MenuName_Cleaned') saved to 'results/snapshots/snapshot_5_after_basic_MenuName_cleaning.html'

--- Initial Data Cleaning & Preprocessing (Part 1 - Basic Steps) Complete ---
Current DataFrame shape: (5674222, 16)
--------------------------------------------------


In [26]:
# --- Step 2.h: Detailed `Site` Column Refinement ---
print("\n--- Step 2.h: Detailed `Site` Column Refinement ---")
if 'Site_Cleaned' in df_processed.columns:
    print("Unique 'Site_Cleaned' values BEFORE detailed refinement:")
    print(df_processed['Site_Cleaned'].unique())

    # Define expected sites and a mapping for common variations
    # This mapping should be expanded based on your actual data exploration
    site_mapping = {
        'ms': 'MS', 'muenster': 'MS', 'münster': 'MS',
        'lp': 'LP', 'lippstadt': 'LP',
        'bk': 'BK', 'bergkamen': 'BK',
        # Add other variations you find to this dictionary
        'unknownsite': 'UnknownSite' # Ensure 'UnknownSite' itself is consistently cased if it came from NaNs
    }

    # Apply mapping: Convert to lowercase first for case-insensitive mapping
    df_processed['Site_Standardized'] = df_processed['Site_Cleaned'].astype(str).str.lower().map(site_mapping).fillna(df_processed['Site_Cleaned'])
    # For values not in mapping, keep original (or map to 'UnknownSite' if preferred)
    # A more robust approach for unmapped values:
    # df_processed['Site_Standardized'] = df_processed['Site_Cleaned'].astype(str).str.lower().map(site_mapping)
    # df_processed['Site_Standardized'].fillna('UnknownSite_Unmapped', inplace=True) # Or some other category

    expected_sites_final = ['MS', 'LP', 'BK', 'UnknownSite'] # Final expected categories

    # Check for any sites not fitting the expected categories after mapping
    current_standardized_sites = df_processed['Site_Standardized'].unique()
    unexpected_standardized_sites = [site for site in current_standardized_sites if site not in expected_sites_final]

    if unexpected_standardized_sites:
        print(f"Warning: Unexpected 'Site_Standardized' values found after mapping: {unexpected_standardized_sites}")
        print("These might need to be added to the site_mapping or handled as a separate category.")
        # For now, let's map remaining unexpected ones to 'UnknownSite' for simplicity in this example
        df_processed.loc[df_processed['Site_Standardized'].isin(unexpected_standardized_sites), 'Site_Standardized'] = 'UnknownSite_Mapped'

    print("\nUnique 'Site_Standardized' values AFTER detailed refinement and mapping:")
    print(df_processed['Site_Standardized'].value_counts())

    snapshot6_path = os.path.join(snapshot_dir, 'snapshot_6_after_Site_Standardization.html')
    df_processed.head(rows_for_snapshot).to_html(snapshot6_path, escape=False, index=False)
    print(f"\nSnapshot 6: First {rows_for_snapshot} rows (showing 'Site_Standardized') saved to '{snapshot6_path}'")
else:
    print("Warning: 'Site_Cleaned' column not found. Skipping detailed Site refinement.")
print("-" * 50)


--- Step 2.h: Detailed `Site` Column Refinement ---
--------------------------------------------------


In [27]:
unique_sites = df_processed['Site'].unique()
print(unique_sites)


['LP' 'BK' 'MS' 'UnknownSite']


In [28]:
# showing datas wich Site in them is unknown
unknown_count = (df_processed['Site'] == 'UnknownSite').sum()
print(f"Number of 'UnknownSite' entries: {unknown_count}")

# 2. Show table with rows that have 'UnknownSite'
unknown_rows = df_processed[df_processed['Site'] == 'UnknownSite']
print("Rows with 'UnknownSite':")
print(unknown_rows)
# snapshot_unknown_path = os.path.join(snapshot_dir, 'snapshot_unknownsite.html')
# df_unknown = df_processed[df_processed['Site'] == 'UnknownSite']
# df_unknown = df[df['Site'] == 'UnknownSite'].head(rows_for_snapshot)


# Save filtered table to HTML
# df_unknown.to_html(snapshot_unknown_path, escape=False, index=False)


snapshot7_path = os.path.join(snapshot_dir, 'snapshot_7_unknownsites.html')
unknown_rows.head(rows_for_snapshot).to_html(snapshot7_path, escape=False, index=False)
print(f"\nSnapshot 7: First {rows_for_snapshot} rows (showing 'Site_Standardized') saved to '{snapshot7_path}'")

Number of 'UnknownSite' entries: 704888
Rows with 'UnknownSite':
          OrderId                           TransactionId DateOfService  \
1193     11212600        d53a378d399c4de8a3d63c82f6f6e9fe    2020-01-07   
1200     11212719        c842569a7de84017ae3bc56a8e2a9c0f    2020-01-07   
1202     11212733        f5486c5957b54098a70c7b75dcd292d2    2020-01-07   
1204     11328252        0c65b7caba4042438cb047a0c37ab68d    2020-01-07   
1217     11212904        79d8f18ab19b4954af0bebbb1f2e7b17    2020-01-07   
...           ...                                     ...           ...   
6536406  21292966  9b50982436e842959afaa7bd71465a6b_order    2023-12-21   
6536412  21053392  f484fca48d884ebc87e3fbe1eda43a8a_order    2023-12-21   
6536503  21558501        ddc7942370f345e0a13fd526bd22aadc    2023-12-21   
6536652  21315597  c664aaff783d43feade8f9e275826c50_order    2023-12-21   
6536827  21256212        e4f7a3719bf84c7da81c44685701457e    2023-12-21   

                DateOfOrder  Order

In [29]:
print(df_processed.head())
snapshot5_path = os.path.join(snapshot_dir, 'snapshot_5_after_basic_MenuName_cleaning.html')
df_processed.head(rows_for_snapshot).to_html(snapshot5_path, escape=False, index=False)

    OrderId                     TransactionId DateOfService  \
0  11518978  4c5060636f584ef9a1effa77282755f5    2020-01-02   
1  11285143  68472c70b9c84fb784834ecc257827d7    2020-01-02   
2  11285146  7262eace0d104592b1269e38f5b45ec1    2020-01-02   
3  11285152  8e451931e8fc4554869c3e4533b65e23    2020-01-02   
4  11285155  bfa8fa0812ee40baa98e5aaf52d30e0b    2020-01-02   

          DateOfOrder  OrderQty                     MenuName  MenuPrice  \
0 2020-02-05 11:54:08         1             Mittagessen (Gs)        310   
1 2019-12-16 10:30:51         1  Smart Eating Buffet (WGrus)          0   
2 2019-12-16 10:31:33         1  Smart Eating Buffet (WGrus)        290   
3 2019-12-16 10:32:05         1  Smart Eating Buffet (WGrus)          0   
4 2019-12-16 10:32:31         1  Smart Eating Buffet (WGrus)        290   

   MenuSubsidy      BookingNr                       GroupName  CanceledQty  \
0            0   349-88220481        xxx3,45€ normal 5T (68€)            0   
1          350

In [30]:
# --- Step 2.i: Advanced `MenuName` Standardization (Iterative Process) ---
print("\n--- Step 2.i: Advanced `MenuName` Standardization ---")
if 'MenuName_Cleaned' in df_processed.columns:
    print(f"Starting advanced MenuName standardization. Unique names so far: {df_processed['MenuName_Cleaned'].nunique()}")

    # Strategy 1: Further Text Normalization (example: remove special characters beyond basic strip/lower)
    # This regex keeps alphanumeric chars, german umlauts, and spaces. Adjust as needed.
    df_processed['MenuName_Normalized'] = df_processed['MenuName_Cleaned'].astype(str).apply(
        lambda x: re.sub(r'[^a-zA-Z0-9äöüÄÖÜß\s]', '', x) # Keep relevant chars
    )
    df_processed['MenuName_Normalized'] = df_processed['MenuName_Normalized'].str.replace(r'\s+', ' ', regex=True).str.strip() # Replace multiple spaces with one

    print(f"Unique names after further normalization (alphanumeric, umlauts, spaces only): {df_processed['MenuName_Normalized'].nunique()}")

    # Strategy 2: Rule-based Mapping (Example - this needs to be built based on your data)
    # This is highly iterative. You'd identify common patterns and map them.
    # Example: (You will need to inspect your data to create meaningful rules)
    menu_name_mapping = {
        'spaghetti bolognese': 'spaghetti bolo',
        'spagetti bolognese': 'spaghetti bolo',
        'spag. bolo': 'spaghetti bolo',
        'currywurst mit pommes': 'currywurst pommes',
        'currywurst m. pommes': 'currywurst pommes',
        'gemüsepfanne vegetarisch': 'gemüsepfanne veg',
        'menü a': 'menü a standard', # Example if 'menü a' has variations
        'menu a': 'menü a standard',
        # ... add many more rules based on your data inspection ...
    }
    # Apply the mapping to the 'MenuName_Normalized' column
    # Create a new column for the mapped names to preserve the previous step
    df_processed['MenuName_Standardized'] = df_processed['MenuName_Normalized'].replace(menu_name_mapping)

    print(f"Unique names after example rule-based mapping: {df_processed['MenuName_Standardized'].nunique()}")

    if df_processed['MenuName_Standardized'].nunique() > 50:
        print("First 20 unique 'MenuName_Standardized' (sample after mapping):")
        print(df_processed['MenuName_Standardized'].unique()[:20])
    else:
        print("Unique 'MenuName_Standardized':")
        print(df_processed['MenuName_Standardized'].unique())

    print("\nIMPORTANT: 'MenuName' standardization is an iterative process.")
    print("You'll likely need to: ")
    print("  1. Analyze frequent variations of 'MenuName_Normalized' or 'MenuName_Standardized'.")
    print("  2. Expand your `menu_name_mapping` dictionary.")
    print("  3. Re-run and check unique counts until they are manageable and meaningful.")
    print("  Consider techniques like grouping by common substrings if appropriate.")

    snapshot7_path = os.path.join(snapshot_dir, 'snapshot_7_after_MenuName_Standardization.html')
    df_processed.head(rows_for_snapshot).to_html(snapshot7_path, escape=False, index=False)
    print(f"\nSnapshot 7: First {rows_for_snapshot} rows (showing 'MenuName_Standardized') saved to '{snapshot7_path}'")

else:
    print("Warning: 'MenuName_Cleaned' column not found. Skipping advanced MenuName standardization.")
print("-" * 50)


print("\n--- Data Cleaning & Preprocessing (Core Steps Completed) ---")
print("Cleaned DataFrame shape (after site and menu name initial standardization):", df_processed.shape)

final_snapshot_path_cleaned = os.path.join(snapshot_dir, 'snapshot_final_cleaned_dataframe.html')
df_processed.head(rows_for_snapshot).to_html(final_snapshot_path_cleaned, escape=False, index=False)
print(f"\nFinal Cleaned Snapshot: First {rows_for_snapshot} rows of the processed DataFrame saved to '{final_snapshot_path_cleaned}'")

print("\nProcessed DataFrame info (final cleaned):")
df_processed.info(verbose=True, show_counts=True)
print("-" * 50)


--- Step 2.i: Advanced `MenuName` Standardization ---
Starting advanced MenuName standardization. Unique names so far: 322
Unique names after further normalization (alphanumeric, umlauts, spaces only): 319
Unique names after example rule-based mapping: 319
First 20 unique 'MenuName_Standardized' (sample after mapping):
['mittagessen gs' 'smart eating buffet wgrus' 'mittagessen bs'
 'smart eating buffet primus' 'menü a hss' 'smart eating buffet gymbo'
 'smart eating buffet brs' 'menü b1' 'smart eating buffet ema'
 'mittagessen ml' 'smarteating buffet egm' 'dgemenü mg' 'menü a bomitte'
 'menü a marbi' 'smart eating buffet gma' 'mittagessen bk'
 'menü a standard' 'menü a szg' 'salatbar mpg' 'menü b2 lui']

IMPORTANT: 'MenuName' standardization is an iterative process.
You'll likely need to: 
  1. Analyze frequent variations of 'MenuName_Normalized' or 'MenuName_Standardized'.
  2. Expand your `menu_name_mapping` dictionary.
  3. Re-run and check unique counts until they are manageable an

In [40]:
df_processed

KeyboardInterrupt: 

In [41]:
print("\n--- Step 3: Feature Engineering ---")

# Ensure DateOfService is datetime
if 'DateOfService' in df_processed.columns and not pd.api.types.is_datetime64_any_dtype(df_processed['DateOfService']):
    df_processed['DateOfService'] = pd.to_datetime(df_processed['DateOfService'], errors='coerce')
    df_processed.dropna(subset=['DateOfService'], inplace=True) # Drop rows where DateOfService became NaT

if 'DateOfService' not in df_processed.columns:
    print("ERROR: 'DateOfService' column is missing. Cannot proceed with feature engineering.")
    exit()

# a. Extract Date/Time Features from DateOfService
print("\nExtracting date/time features from 'DateOfService'...")
df_processed['Year'] = df_processed['DateOfService'].dt.year
df_processed['Month'] = df_processed['DateOfService'].dt.month
df_processed['DayOfMonth'] = df_processed['DateOfService'].dt.day
df_processed['DayOfWeek'] = df_processed['DateOfService'].dt.dayofweek # Monday=0, Sunday=6
df_processed['DayName'] = df_processed['DateOfService'].dt.day_name()
df_processed['DayOfYear'] = df_processed['DateOfService'].dt.dayofyear
df_processed['WeekOfYear'] = df_processed['DateOfService'].dt.isocalendar().week.astype(int)
df_processed['Quarter'] = df_processed['DateOfService'].dt.quarter
df_processed['IsWeekend'] = df_processed['DayOfWeek'].isin([5, 6]).astype(int) # Saturday or Sunday

print("Date/time features created: Year, Month, DayOfMonth, DayOfWeek, DayName, DayOfYear, WeekOfYear, Quarter, IsWeekend")

# b. Order Lead Time Features (relative to DateOfService)
print("\nCreating order lead time features...")
if 'DateOfOrder' in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed['DateOfOrder']):
    df_processed['OrderLeadTimeDays'] = (df_processed['DateOfService'] - df_processed['DateOfOrder']).dt.days
    # Categorize lead time based on use case owner's benchmarks
    bins = [-float('inf'), 1, 5, float('inf')] # Bins: <=1 day, 2-5 days, >5 days
    labels = ['1_day_or_less', '2_to_5_days', 'gt_5_days']
    df_processed['OrderLeadTimeCategory'] = pd.cut(df_processed['OrderLeadTimeDays'], bins=bins, labels=labels, right=True)
    print("'OrderLeadTimeDays' and 'OrderLeadTimeCategory' created.")
    print(df_processed['OrderLeadTimeCategory'].value_counts(dropna=False))
else:
    print("Warning: 'DateOfOrder' not found or not datetime. Skipping OrderLeadTime features.")

# c. Cancellation Lead Time Features (relative to DateOfService)
print("\nCreating cancellation lead time features...")
if 'DateOfCancel' in df_processed.columns and pd.api.types.is_datetime64_any_dtype(df_processed['DateOfCancel']):
    df_processed['CancelLeadTimeDays'] = (df_processed['DateOfService'] - df_processed['DateOfCancel']).dt.days
    # Note: DateOfCancel can be NaT if not canceled. CancelLeadTimeDays will be NaN for these.
    # We can categorize these NaNs as 'NotCanceled' if useful.
    df_processed['CancelLeadTimeCategory'] = pd.cut(df_processed['CancelLeadTimeDays'], bins=bins, labels=labels, right=True)
    # For orders that were not canceled, DateOfCancel is NaT, so CancelLeadTimeDays is NaT.
    # Let's fill the category for these as 'NotCanceled'.
    df_processed['CancelLeadTimeCategory'] = df_processed['CancelLeadTimeCategory'].cat.add_categories(['NotCanceled']).fillna('NotCanceled')

    print("'CancelLeadTimeDays' and 'CancelLeadTimeCategory' created.")
    print(df_processed['CancelLeadTimeCategory'].value_counts(dropna=False))
else:
    print("Warning: 'DateOfCancel' not found or not datetime. Skipping CancelLeadTime features.")


# d. Holiday Features (Placeholder - Prophet handles this well during modeling)
print("\nHoliday Features Consideration:")
print("Prophet has built-in functionality for holidays (country-specific or custom).")
print("This will be addressed during the Prophet model setup for each time series.")
print("For Germany: Prophet can use add_country_holidays(country_name='DE').")


# e. Interaction Features or Other Derived Features (Examples)
# print("\nConsidering other derived features (examples):")
# Example: Is it a Monday after a holiday weekend? (More complex, requires holiday data)
# Example: Price per meal (if OrderQty > 0)
# if 'MenuPrice' in df_processed.columns and 'OrderQty' in df_processed.columns:
#     df_processed['PricePerUnit'] = df_processed['MenuPrice'] / df_processed['OrderQty'].replace(0, np.nan) # Avoid division by zero

print("\nSnapshot after Feature Engineering:")
snapshot_fe_path = os.path.join(snapshot_dir, 'snapshot_FE_dataframe.html') # Assuming snapshot_dir is defined
df_processed.head(rows_for_snapshot).to_html(snapshot_fe_path, escape=False, index=False)
print(f"First {rows_for_snapshot} rows of DataFrame with new features saved to '{snapshot_fe_path}'")
print(df_processed.info(verbose=False, show_counts=True))
print("-" * 50)



--- Step 3: Feature Engineering ---

Extracting date/time features from 'DateOfService'...
Date/time features created: Year, Month, DayOfMonth, DayOfWeek, DayName, DayOfYear, WeekOfYear, Quarter, IsWeekend

Creating order lead time features...
'OrderLeadTimeDays' and 'OrderLeadTimeCategory' created.
OrderLeadTimeCategory
gt_5_days        3961201
1_day_or_less    1290896
2_to_5_days       422125
Name: count, dtype: int64

Creating cancellation lead time features...
'CancelLeadTimeDays' and 'CancelLeadTimeCategory' created.
CancelLeadTimeCategory
NotCanceled      5604260
gt_5_days          62519
2_to_5_days         5054
1_day_or_less       2389
Name: count, dtype: int64

Holiday Features Consideration:
Prophet has built-in functionality for holidays (country-specific or custom).
This will be addressed during the Prophet model setup for each time series.
For Germany: Prophet can use add_country_holidays(country_name='DE').

Snapshot after Feature Engineering:
First 1000 rows of DataFrame

In [42]:
df_processed

,OrderId,TransactionId,DateOfService,DateOfOrder,OrderQty,MenuName,MenuPrice,MenuSubsidy,BookingNr,GroupName,...,DayOfWeek,DayName,DayOfYear,WeekOfYear,Quarter,IsWeekend,OrderLeadTimeDays,OrderLeadTimeCategory,CancelLeadTimeDays,CancelLeadTimeCategory
0,11518978,4c5060636f584ef9a1effa77282755f5,2020-01-02,2020-02-05 11:54:08,1,Mittagessen (Gs),310,0,349-88220481,"xxx3,45€ normal 5T (68€)",...,3,Thursday,2,1,1,0,-35,1_day_or_less,NaN,NotCanceled
1,11285143,68472c70b9c84fb784834ecc257827d7,2020-01-02,2019-12-16 10:30:51,1,Smart Eating Buffet (WGrus),0,350,248-141751492,Steinfurt Abo ermäßigt,...,3,Thursday,2,1,1,0,16,gt_5_days,NaN,NotCanceled
2,11285146,7262eace0d104592b1269e38f5b45ec1,2020-01-02,2019-12-16 10:31:33,1,Smart Eating Buffet (WGrus),290,60,248-77489928,Westerkappeln Grundschüler Abo,...,3,Thursday,2,1,1,0,16,gt_5_days,NaN,NotCanceled
3,11285152,8e451931e8fc4554869c3e4533b65e23,2020-01-02,2019-12-16 10:32:05,1,Smart Eating Buffet (WGrus),0,350,248-77558043,Steinfurt Abo ermäßigt,...,3,Thursday,2,1,1,0,16,gt_5_days,NaN,NotCanceled
4,11285155,bfa8fa0812ee40baa98e5aaf52d30e0b,2020-01-02,2019-12-16 10:32:31,1,Smart Eating Buffet (WGrus),290,60,248-77420774,Westerkappeln Grundschüler Abo,...,3,Thursday,2,1,1,0,16,gt_5_days,NaN,NotCanceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6538732,21565890,fd46957648df48d9bd95f785357cbe25_order,2023-12-22,2023-12-18 08:50:21,1,Mittagessen (KL),370,0,349-167443917,"3,70 €",...,4,Friday,356,51,4,0,3,2_to_5_days,NaN,NotCanceled
6538734,20946470,c66797e99f7d4684a249d2fde8e03d4d_order,2023-12-22,2023-11-14 15:11:29,1,Mittagessen,0,375,721-161089197,"BuT (E,P)",...,4,Friday,356,51,4,0,37,gt_5_days,NaN,NotCanceled
6538735,21178498,cf8caf00f8fc4674ac8039562275c0fd_order,2023-12-22,2023-11-27 12:11:24,1,Mittagessen (KL),370,0,349-163628248,"3,70 €",...,4,Friday,356,51,4,0,24,gt_5_days,NaN,NotCanceled
6538736,21178551,ade8ad875b904ef89fb82aefb4d089ea_order,2023-12-22,2023-11-27 12:11:31,1,Mittagessen (KL),370,0,349-165661318,"3,70 €",...,4,Friday,356,51,4,0,24,gt_5_days,NaN,NotCanceled
